In [1]:
#Libraries
import glob
import json
import os
import re
import string

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing

# Real Estate Data

We will analyze real estate data from a real estate website.

## Data Ingestion

In [10]:
#Reading file
newFile = '/home/DAITAN/ftsantos/workingdir/pro/bitbucket/workingdir/per/git/lab/projects/realestate/data/immobilienscout24_berlin_20190113.json'

#Parsing Json
dfNew = pd.read_json(path_or_buf=newFile, typ='frame', orient='values', dtype=False, convert_dates=False, lines=True)        

## Analyzing the dataset

In this section there are just some small analysis to understand the dataset.

In [13]:
dfNew.shape

(3596, 2)

In [16]:
#dfNew.describe()

In [15]:
dfNew.columns

Index(['data', 'ok'], dtype='object')

In [17]:
dfNew.dtypes

data    object
ok        bool
dtype: object

In [18]:
dfNew.head()

,data,ok
0,"{'contactFormType': 'LONG', 'creation': '2017-...",True
1,"{'contactFormType': 'EMAIL', 'creation': '2019...",True
2,"{'contactFormType': 'EMAIL', 'creation': '2018...",True
3,"{'contactFormType': 'EMAIL', 'creation': '2019...",True
4,"{'contactFormType': 'LONG', 'creation': '2018-...",True


## Data Prep

The data needs to be prepared in order to be used for further analysis.

In [19]:
#Expands the Data field
dfData = dfNew.data.apply(pd.Series)

In [20]:
dfData.head()

,adLinkForJSONP_xlink_href,adLinkForXMLData_xlink_href,companyWideCustomerId,contactDetails_address_city,contactDetails_address_houseNumber,contactDetails_address_postcode,contactDetails_address_street,contactDetails_cellPhoneNumber,contactDetails_cellPhoneNumberAreaCode,contactDetails_cellPhoneNumberCountryCode,...,realEstate_titlePicture_urls,realEstate_totalRent,realEstate_usableFloorSpace,realEstate_useAsFlatshareRoom,realEstate_xsi_type,realtorValuationJSONLink_xlink_href,realtorValuationV2JSONLink_xlink_href,realtorValuationV2JSONPLink_xlink_href,showcasePlacementColor,xlink_href
0,pubads.g.doubleclick.net/gampad/adx?iu=/4467/I...,https://pubads.g.doubleclick.net/gampad/adx?iu...,001.377811,Berlin,56,14193,Koenigsallee,0172 303 60 94,172,+49,...,"[{'url': [{'@scale': 'SCALE_60x60', '@href': '...",2115.19,NaN,NOT_APPLICABLE,expose:ApartmentRent,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,NaN,https://rest.immobilienscout24.de/restapi/api/...
1,pubads.g.doubleclick.net/gampad/adx?iu=/4467/I...,https://pubads.g.doubleclick.net/gampad/adx?iu...,001.2212140,,,,,,NaN,NaN,...,"[{'url': [{'@scale': 'SCALE_60x60', '@href': '...",437.88,31.32,NOT_APPLICABLE,expose:ApartmentRent,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,NaN,https://rest.immobilienscout24.de/restapi/api/...
2,pubads.g.doubleclick.net/gampad/adx?iu=/4467/I...,https://pubads.g.doubleclick.net/gampad/adx?iu...,001.634246,Berlin,NaN,NaN,NaN,,NaN,NaN,...,NaN,413.00,37.67,NOT_APPLICABLE,expose:ApartmentRent,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,NaN,https://rest.immobilienscout24.de/restapi/api/...
3,pubads.g.doubleclick.net/gampad/adx?iu=/4467/I...,https://pubads.g.doubleclick.net/gampad/adx?iu...,001.12610602,,,,,,NaN,NaN,...,"[{'url': [{'@scale': 'SCALE_60x60', '@href': '...",850.00,75.30,NOT_APPLICABLE,expose:ApartmentRent,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,NaN,https://rest.immobilienscout24.de/restapi/api/...
4,pubads.g.doubleclick.net/gampad/adx?iu=/4467/I...,https://pubads.g.doubleclick.net/gampad/adx?iu...,001.717259,Berlin,36,12207,Königsberger Straße,,NaN,NaN,...,"[{'url': [{'@scale': 'SCALE_60x60', '@href': '...",1024.50,56.50,NOT_APPLICABLE,expose:ApartmentRent,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,https://www.immobilienscout24.de/expose/realto...,NaN,https://rest.immobilienscout24.de/restapi/api/...


In [37]:
#Seeing all columns
aux = 0
for col in dfData.columns:
    print (dfData.columns[aux])
    aux = aux + 1

adLinkForJSONP_xlink_href
adLinkForXMLData_xlink_href
companyWideCustomerId
contactDetails_address_city
contactDetails_address_houseNumber
contactDetails_address_postcode
contactDetails_address_street
contactDetails_cellPhoneNumber
contactDetails_cellPhoneNumberAreaCode
contactDetails_cellPhoneNumberCountryCode
contactDetails_cellPhoneNumberSubscriber
contactDetails_company
contactDetails_countryCode
contactDetails_email
contactDetails_faxNumber
contactDetails_faxNumberAreaCode
contactDetails_faxNumberCountryCode
contactDetails_faxNumberSubscriber
contactDetails_firstname
contactDetails_homepageUrl
contactDetails_id
contactDetails_lastname
contactDetails_phoneNumber
contactDetails_phoneNumberAreaCode
contactDetails_phoneNumberCountryCode
contactDetails_phoneNumberSubscriber
contactDetails_portraitUrl
contactDetails_realtorLogo
contactDetails_salutation
contactDetails_title
contactFormConfiguration_addressField
contactFormConfiguration_applicationPackageCompletedField
contactFormConfigu

In [44]:
dfData.dtypes

adLinkForJSONP_xlink_href                     object
adLinkForXMLData_xlink_href                   object
companyWideCustomerId                         object
contactDetails_address_city                   object
contactDetails_address_houseNumber            object
contactDetails_address_postcode               object
contactDetails_address_street                 object
contactDetails_cellPhoneNumber                object
contactDetails_cellPhoneNumberAreaCode        object
contactDetails_cellPhoneNumberCountryCode     object
contactDetails_cellPhoneNumberSubscriber      object
contactDetails_company                        object
contactDetails_countryCode                    object
contactDetails_email                          object
contactDetails_faxNumber                      object
contactDetails_faxNumberAreaCode              object
contactDetails_faxNumberCountryCode           object
contactDetails_faxNumberSubscriber            object
contactDetails_firstname                      